In [1]:
!pip -q install faiss-cpu sentence-transformers tqdm


In [2]:
from pathlib import Path
import json
import numpy as np
import faiss
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

INDEX_FILE = Path("vector_db/iea_faiss.index")
META_FILE  = Path("vector_db/iea_metadata.jsonl")

assert INDEX_FILE.exists(), "FAISS index not found. Run vector_store.ipynb first."
assert META_FILE.exists(), "Metadata file not found. Run vector_store.ipynb first."

index = faiss.read_index(str(INDEX_FILE))
print("✅ Loaded FAISS index | vectors:", index.ntotal)

metadata = []
with META_FILE.open("r", encoding="utf-8") as f:
    for line in f:
        metadata.append(json.loads(line))

print("✅ Loaded metadata records:", len(metadata))


/Users/adityaamiddha/hello/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Loaded FAISS index | vectors: 352
✅ Loaded metadata records: 352


In [3]:
MODEL_NAME = "BAAI/bge-small-en-v1.5"
embedder = SentenceTransformer(MODEL_NAME)
print("✅ Loaded embedder:", MODEL_NAME)


✅ Loaded embedder: BAAI/bge-small-en-v1.5


In [4]:
def retrieve(query: str, k: int = 5):
    qv = embedder.encode(query, normalize_embeddings=True).astype("float32")
    D, I = index.search(np.array([qv]), k)
    
    results = []
    for score, idx in zip(D[0], I[0]):
        if idx == -1:
            continue
        item = metadata[idx].copy()
        item["score"] = float(score)
        results.append(item)
    return results


In [5]:
def show_results(query, k=5, max_chars=450):
    print("\n" + "="*110)
    print("QUERY:", query)
    print("="*110)

    results = retrieve(query, k=k)
    for r, item in enumerate(results, 1):
        snippet = item["text"][:max_chars].replace("\n", " ")
        print(f"\n#{r}  score={item['score']:.4f} | {item['chunk_id']} | year={item['year']} | source={item['source']}")
        print("    ", snippet, "...")


In [6]:
show_results("How have global EV sales changed in 2022 and 2023?", k=5)



QUERY: How have global EV sales changed in 2022 and 2023?

#1  score=0.8401 | IEA_2023_000010 | year=2023 | source=GEVO2023_clean
     is expected to expand. Box 1.1 The 2023 outlook for electric cars is bright Early indications from first quarter sales of 2023 point to an upbeat market, supported by cost declines as well as strengthened policy support in key markets such as the United States. Globally, our current estimate is therefore for nearly 14 million electric cars to be sold in 2023, building on the more than 2.3 million already sold in the first quarter of the year. Thi ...

#2  score=0.8366 | IEA_2024_000008 | year=2024 | source=GlobalEVOutlook2024_clean
     include all modes of road transport. 5 15 25 35 45 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 Million Rest of the world PHEV Rest of the world BEV United States PHEV United States BEV Europe PHEV Europe BEV China PHEV China BEV Global EV Outlook 2024 Trends in electric cars PAGE | 18 IEA. CC B

In [7]:
def paraphrase_test(queries, k=5):
    for q in queries:
        show_results(q, k=k)


In [8]:
queries = [
    "Global EV sales trend in 2023",
    "How did electric car sales grow in 2023?",
    "Tell me the worldwide EV sales statistics for 2023",
    "What is the EV market share growth globally in 2023?"
]

paraphrase_test(queries, k=5)



QUERY: Global EV sales trend in 2023

#1  score=0.8782 | IEA_2023_000010 | year=2023 | source=GEVO2023_clean
     is expected to expand. Box 1.1 The 2023 outlook for electric cars is bright Early indications from first quarter sales of 2023 point to an upbeat market, supported by cost declines as well as strengthened policy support in key markets such as the United States. Globally, our current estimate is therefore for nearly 14 million electric cars to be sold in 2023, building on the more than 2.3 million already sold in the first quarter of the year. Thi ...

#2  score=0.8650 | IEA_2023_000007 | year=2023 | source=GEVO2023_clean
     which has a 50% NEV sales share by 2025 target, facilitated the expansion of Xpeng Motors to become one of the national EV frontrunners. 3 NEVs (China) include BEVs, PHEVs and fuel cell electric vehicles. Global EV Outlook 2023 Trends and developments in EV markets Catching up with climate ambitions PAGE | 17 IEA. CC BY 4.0. Figure 1.2 Monthly new ele

In [9]:
charging_queries = [
    "How many public charging points exist globally?",
    "Fast chargers vs slow chargers growth",
    "Charging infrastructure investment needs"
]
paraphrase_test(charging_queries, k=5)



QUERY: How many public charging points exist globally?

#1  score=0.7928 | IEA_2025_000055 | year=2025 | source=GlobalEVOutlook2025_clean
     to 150 kW as fast, and 150 kW and above as ultra-fast. Sources: IEA analysis based on country submissions and data from EV Volumes and EAFO. Brazil's charging infrastructure has expanded at speed. By early December 2024, there were over 12 000 public charging points across the country. Public charging networks have also experienced rapid expansion in other key emerging markets, reflecting strong policy support and investment in infrastructure. Pu ...

#2  score=0.7697 | IEA_2025_000063 | year=2025 | source=GlobalEVOutlook2025_clean
     2024 to over 500 000 at the end of 2030 in the STEPS. This assumes the historic trend in the increasing ratio of electric LDV stock per public charging point continues, and that in 2030 there are almost 40 electric LDVs per public charging point, up from around 32 in 2024. Although this ratio is significantly hi

In [10]:
battery_queries = [
    "Battery demand growth in GWh",
    "Lithium and nickel supply chain constraints",
    "Battery manufacturing capacity trends"
]
paraphrase_test(battery_queries, k=5)



QUERY: Battery demand growth in GWh

#1  score=0.8046 | IEA_2022_000056 | year=2022 | source=GlobalElectricVehicleOutlook2022_clean
     supply chains Battery demand for EVs doubled in 2021 Automotive lithium-ion (Li-ion) battery demand was 340 gigawatt-hours (GWh) in 2021, more than twice the level of 2020. This increase is driven by the increase in electric passenger cars (registrations increased by 120%). The average battery capacity of battery electric vehicles (BEVs) was 55 kilowatt-hours (kWh) in 2021, down from 56 kWh in 2020, whereas the average capacity increased for plug ...

#2  score=0.7881 | IEA_2022_000072 | year=2022 | source=GlobalElectricVehicleOutlook2022_clean
     and is especially pronounced in North America. China is projected to have the largest battery demand, though its global share shrinks from 60% in 2021 to 40% in 2030 in the Stated Policies Scenario and 25% in the Announced Pledges Scenario. The United States undergoes the fastest battery demand increase a

In [11]:
policy_queries = [
    "Which policies are accelerating EV adoption?",
    "EV subsidies and incentives globally",
    "Government support for charging infrastructure"
]
paraphrase_test(policy_queries, k=5)



QUERY: Which policies are accelerating EV adoption?

#1  score=0.8104 | IEA_2023_000034 | year=2023 | source=GEVO2023_clean
     some governments in major markets have increased their targets for EV adoption further and are working to address other parts of EV supply chains, such as through policy support for vehicle and battery manufacturing and critical mineral supply chains. Many other countries outside the major markets have also started to introduce policy to support EV adoption in recent years, for the first time in some cases. Overall, global spending by governments ...

#2  score=0.7829 | IEA_2024_000007 | year=2024 | source=GlobalEVOutlook2024_clean
     residential and workplace settings. To reach EV deployment levels in the Announced Policies Scenario, public charging needs to increase sixfold by 2035. As more electric heavy-duty vehicles such as trucks and large buses hit the road, dedicated and flexible charging is needed. In 2023, electric buses accounted for 3% of total

In [12]:
test_query = "What are the major EV adoption trends discussed in Global EV Outlook?"
for k in [3, 5, 8, 10]:
    print("\n\n" + "#"*40, f"TOP K = {k}", "#"*40)
    show_results(test_query, k=k)




######################################## TOP K = 3 ########################################

QUERY: What are the major EV adoption trends discussed in Global EV Outlook?

#1  score=0.8336 | IEA_2023_000034 | year=2023 | source=GEVO2023_clean
     some governments in major markets have increased their targets for EV adoption further and are working to address other parts of EV supply chains, such as through policy support for vehicle and battery manufacturing and critical mineral supply chains. Many other countries outside the major markets have also started to introduce policy to support EV adoption in recent years, for the first time in some cases. Overall, global spending by governments ...

#2  score=0.8260 | IEA_2025_000044 | year=2025 | source=GlobalEVOutlook2025_clean
     series. Global EV Outlook 2025 Outlook for electric mobility PAGE | 82 IEA. CC BY 4.0. 5. Outlook for electric mobility Overview In this part of the report, we focus on the outlook for electric mobility in ro